# BigQuery Sample Codes  
DataFabric 분석환경에서 제공하는 JupyerHub에서 pydatafabric 파이썬 패키지를 사용하면 BigQuery와 연동하여 분석 및 모델링 작업을 수행할 수 있습니다.
pydatafabric 패키지는 분석환경에 기본적으로 설치되어 있으며 pip를 사용하여 버전 업그레이드를 할 수 있습니다.
~~~bash
$ pip install --upgrade pydatafabric
~~~

## BigQuery 실행  
다음과 같이 2가지 방법으로 쿼리를 실행하고 결과를 확인할 수 있습니다.  
- IPython Magic을 사용하여 Jupyter Notebook Cell에서 SQL 실행
- BigQuery Client 사용하여 SQL 실행

### IPython Magic으로 SQL 실행  
Jupyter Notebook Cell에서 SQL을 실행하고 그 결과를 확인할 수 있습니다. 쿼리 결과는 변수에 Pandas Dataframe으로 저장할 수 있습니다. Ditonary로 정의된 쿼리 파라미터를 SQL에 주입할 수 있습니다.

먼저, IPython Magic을 로드합니다.

In [14]:
PROJECT = "emart-datafabric"
DATASET = "common_dev"
TABLE = "dfm_sample_eapp_data"
LIMIT = 30

In [15]:
from pydatafabric.gcp import load_bigquery_ipython_magic

load_bigquery_ipython_magic()

다음과 같이 SQL을 실행합니다.

In [2]:
%%bigquery
    SELECT MAX(dt) as max_dt
    FROM emart-datafabric.common_dev.dfm_sample_eapp_data

Downloading: 100%|██████████| 1/1 [00:00<00:00,  1.05rows/s]

BigQuery execution took 3 seconds.


,max_dt
0,2022-08-30


SQL 결과를 Pandas DataFrame으로 변수에 저장할 수 있습니다. 다음은 max_dt라는 변수에 SQL 결과를 저장하는 예제입니다.

In [16]:
%load_ext google.cloud.bigquery

The google.cloud.bigquery extension is already loaded. To reload it, use:
  %reload_ext google.cloud.bigquery


In [17]:
%%bigquery max_dt

SELECT MAX(dt) as value
FROM emart-datafabric.common_dev.dfm_sample_eapp_data

Downloading: 100%|██████████| 1/1 [00:00<00:00,  1.19rows/s]


In [18]:
max_dt

,value
0,2022-08-30


쿼리 파라미터를 SQL에 전달할 수 있습니다. 전달할 값은 Dictionary 타입으로 정의합니다.

In [19]:
query_params = {
    "max_dt": max_dt["value"][0].strftime("%Y-%m-%d")
}
query_params

{'max_dt': '2022-08-30'}

다음과 같이 params 옵션에 전달할 쿼리 파라미터 변수를 정의합니다.

In [20]:
%%bigquery --params $query_params

SELECT *
FROM emart-datafabric.common_dev.dfm_sample_eapp_data
WHERE dt = @max_dt

Downloading: 100%|██████████| 25608/25608 [00:00<00:00, 29877.00rows/s]


,review_id,cust_id,gender_cd,gender_nm,age,agrde_cd_10_unit,store_cd,store_nm,score,comments,...,longitude,latitude,comments_point,image_point,thumb_point,action_cd,blind_flag,active_flag,tag_list,avg_tag_score
0,2208301432069315,02b7d95c81e0a0b16905e77547ac0b54881a0728438961...,2,여,62,60,1000,이마트 창동점,10,치즈 좋아하는데 쟁여 놔야 맘이 편안 그래서 또구입,...,127.046690683,37.651608368,10,10,0,003,N,Y,"332::1::8,332::2::6,332::3::6,332::4::6,332::5::6",6.000000000
1,2208301435131475,02b7d95c81e0a0b16905e77547ac0b54881a0728438961...,2,여,62,60,1000,이마트 창동점,10,모짜렐라 쟁여 놓은게 마침 떨어져서 구입 든든,...,127.046690683,37.651608368,10,10,0,003,N,Y,"332::1::6,332::2::6,332::3::6,332::4::6,332::5::4",6.000000000
2,2208301430288840,02b7d95c81e0a0b16905e77547ac0b54881a0728438961...,2,여,62,60,1000,이마트 창동점,10,담백하고 비리지 않은 가자미 국산이라네요 맛있게 먹을께용,...,127.046690683,37.651608368,10,10,0,003,N,Y,"153::1::6,153::2::6,153::3::6,153::4::6,153::5::6",6.000000000
3,2208300923271031,03c4f22b4d5b998916c8ae5358c68f9d1b89ff96a93bc4...,1,남,36,30,1000,이마트 창동점,10,저렴하고 맛있어요!!!!!!!!!,...,127.046690683,37.651608368,10,10,0,003,N,Y,"130::1::10,130::2::10,130::3::10,130::4::10,13...",10.000000000
4,2208300925053511,03c4f22b4d5b998916c8ae5358c68f9d1b89ff96a93bc4...,1,남,36,30,1000,이마트 창동점,10,맛있어요!!!!!!!!!,...,127.046690683,37.651608368,10,10,0,003,N,Y,"340::1::10,340::2::10,340::3::10,340::4::10,34...",10.000000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25603,2208301158538227,553497183b155892de10e30c28381f06406ce27f457341...,2,여,42,40,7407,토이킹덤 안성점,10,,...,127.147249595,36.994633421,0,0,0,000,N,Y,"542::1::10,542::2::10,542::3::10,542::4::10",10.000000000
25604,2208301522157501,14611f676a02617050fd2484f8ade5c8b9292111ea5e5e...,1,남,51,50,7709,일렉트로마트 스타필드 하남점,10,,...,127.223738447,37.545436142,0,0,0,000,N,Y,"534::1::10,534::2::10,534::3::6,534::4::8",9.000000000
25605,2208300440306896,ec8c3f6f08da9871739cd9273b886b001994de33b5167c...,1,남,48,40,7709,일렉트로마트 스타필드 하남점,10,,...,127.223738447,37.545436142,0,0,0,000,N,Y,"530::1::10,530::2::10,530::3::10,530::4::10",10.000000000
25606,2208300915461856,b4090107c89a0eb834d47e9a35fb80b74993db103bf1c0...,2,여,33,30,7714,일렉트로마트 스타필드 고양점,10,디자인 이쁘고 좋아요,...,126.894776182,37.646978959,10,0,0,001,N,Y,"530::1::10,530::2::10,530::3::10,530::4::10",10.000000000


### BigQuery Client로 SQL 실행  
BigQuery Client는 Google에서 제공하는 Client Library에서 제공하는 객체입니다. SQL을 실행할 수 있을 뿐만 아니라 Google Client Library에서 제공하는 다양한 기능들을 활용할 수 있습니다. 더욱 자세한 내용은 <a href="https://googleapis.dev/python/bigquery/latest/generated/google.cloud.bigquery.client.Client.html#google.cloud.bigquery.client.Client" target="_blank">이곳</a>에서 확인하시기 바랍니다.

다음과 같이 BigQuery Client를 생성하여 SQL을 실행합니다. 쿼리 조회 결과가 있는 경우 Iterator 형태로 결과를 리턴해줍니다.

In [21]:
from pydatafabric.gcp import get_bigquery_client

bq_client = get_bigquery_client()
max_dt = query_params['max_dt']

sql = f"""
    SELECT *
    FROM {PROJECT}.{DATASET}.{TABLE}
    WHERE dt = '{max_dt}'
    LIMIT 5
""".format(**query_params)

iterator = bq_client.query(sql).result()
for r in iterator:
    print(r)

Row(('2208301432069315', '02b7d95c81e0a0b16905e77547ac0b54881a0728438961c0524fc6666c200663', '2', '여', '62', '60', '1000', '이마트 창동점', 10, '치즈 좋아하는데 쟁여 놔야 맘이 편안 그래서 또구입', 28, '8801069416113', '피코크 에이클래스 체다 치즈 210g(30gX7)', datetime.date(2022, 8, 29), datetime.date(2022, 8, 30), '33', '가공B', '30', '가공담당', '332', '유가공품', '0087', 'PEACOCK유가공품', '1050', 'P)치즈', '00', '서울', Decimal('127.046690683'), Decimal('37.651608368'), 10, 10, 0, '003', 'N', 'Y', '332::1::8,332::2::6,332::3::6,332::4::6,332::5::6', Decimal('6')), {'review_id': 0, 'cust_id': 1, 'gender_cd': 2, 'gender_nm': 3, 'age': 4, 'agrde_cd_10_unit': 5, 'store_cd': 6, 'store_nm': 7, 'score': 8, 'comments': 9, 'comments_length': 10, 'sku_cd': 11, 'sku_nm': 12, 'order_dt': 13, 'dt': 14, 'prdt_cat_cd': 15, 'prdt_cat_nm': 16, 'prdt_di_cd': 17, 'prdt_di_nm': 18, 'prdt_gcode_cd': 19, 'prdt_gcode_nm': 20, 'prdt_mcode_cd': 21, 'prdt_mcode_nm': 22, 'prdt_dcode_cd': 23, 'prdt_dcode_nm': 24, 'area_cd': 25, 'area_nm': 26, 'longitude': 27, 'lati

### INSERT OVERWRITE
BigQuery는 Hive와 달리 INSERT OVERWRITE 기능을 제공하지 않습니다. 그러나 emart 패키지의 bq_insert_overwrite 메서드를 사용하면 INSERT OVERWRITE가 가능합니다.  
만약 테이블이 존재하지 않는다면 새로 테이블을 생성합니다.  
저장하려는 대상 테이블이 파티셔닝되어 있다면 partition 파라미터를 사용하여 파티션 컬럼 이름을 설정합니다. sql 결과의 파티션 컬럼 값에 따라 해당 파티션으로 저장됩니다.  
대상 테이블이 cluster 설정이 되어 있거나 새로 생성하는 테이블에 clustrer를 설정하고 싶은 경우 clustering_fields 파라미터에 컬럼 이름을 리스트로 넣어줍니다.

In [22]:
from pydatafabric.gcp import bq_insert_overwrite, get_temp_table

sql = f"""
    SELECT *
    FROM {PROJECT}.{DATASET}.{TABLE}
    WHERE dt = '{max_dt}'
""".format(**query_params)

result_table_name= get_temp_table()

bq_insert_overwrite(sql=sql, destination=result_table_name, partition="dt", clustering_fields=["cust_id"])

destination: emart-datafabric.temp_1d.58ef5e43_c35f_4455_ba70_6d01d8e2046e
total_rows: 25608
slot_secs: 3.323



### Multi Partitions
BigQuery는 단일 컬럼 파티션만 제공합니다. BigQuery에서 멀티 파티션 컬럼을 사용할 수는 없지만 BigQuery에서 제공하는 와일드카드 테이블을 응용하여 멀티 파티션 테이블처럼 정의할 수 있습니다.  
<b>"__"</b>를 구분자로 한 테이블들을 생성하여 멀티 파티션처럼 사용하는 예제입니다.

In [23]:
%%bigquery --params $query_params

CREATE OR REPLACE TABLE emart-datafabric.temp_1d.wildcard_table__subpart_1
AS
SELECT *
FROM emart-datafabric.common_dev.dfm_sample_eapp_data
WHERE dt = @max_dt
;

CREATE OR REPLACE TABLE emart-datafabric.temp_1d.wildcard_table__subpart_2
AS
SELECT *
FROM emart-datafabric.common_dev.dfm_sample_eapp_data
WHERE dt = @max_dt
;

SELECT *
FROM `emart-datafabric.temp_1d.wildcard_table__*`
WHERE _TABLE_SUFFIX = 'subpart_1'
LIMIT 5
;

Downloading: 100%|██████████| 5/5 [00:00<00:00,  6.03rows/s]


,review_id,cust_id,gender_cd,gender_nm,age,agrde_cd_10_unit,store_cd,store_nm,score,comments,...,longitude,latitude,comments_point,image_point,thumb_point,action_cd,blind_flag,active_flag,tag_list,avg_tag_score
0,2208300828272912,7ea94cb7154eac653b5a1eda89907270b65cbc4c6c2b9b...,2,여,54,50,1000,이마트 창동점,10,ㄴㄱㅅㅇㅈ뒥ㅂㅅㅇㅈㅅ노,...,127.046690683,37.651608368,10,0,0,001,Y,Y,"342::1::10,342::2::10,342::3::10,342::4::10,34...",10.000000000
1,2208300707556891,86d736498822620ee8ad5a6228627caebccc73fbcfd801...,2,여,48,40,1000,이마트 창동점,10,아이가좋아해요~~~,...,127.046690683,37.651608368,10,0,0,001,N,Y,"340::1::10,340::2::10,340::3::10,340::4::10,34...",10.000000000
2,2208300006452516,239ec5061e62ff2d3411da43367d84f0f4a8e29b90f06c...,2,여,87,70,1000,이마트 창동점,10,오픈런까지 한 보람은 좀 ..,...,127.046690683,37.651608368,10,10,0,003,N,Y,"154::1::10,154::2::10,154::3::10,154::4::8,154...",10.000000000
3,2208300008031494,b5255a4eb0e00995a1ab4c560b91913cb487cdd2cb64fc...,2,여,56,50,1000,이마트 창동점,10,가격 대비 만족스러웠음~,...,127.046690683,37.651608368,10,0,0,001,N,Y,"154::1::10,154::2::10,154::3::10,154::4::10,15...",10.000000000
4,2208300002169167,3de9a96669656f3cd0202a0fca3bede30ed20614bb9dd7...,2,여,56,50,1000,이마트 창동점,10,조미용 간장이라 편하게 맛있게 이용합니다,...,127.046690683,37.651608368,10,0,0,001,N,Y,"310::1::8,310::2::10,310::3::10,310::4::10,310...",10.000000000


이렇게 와일드카드로 생성한 테이블들에도 INSERT OVERWRITE를 사용할 수 있습니다. emart 패키지의 bq_insert_overwrite_with_suffixes 메서드를 사용하여 특정 suffix를 가지는 테이블의 특정 파티션에 INSERT OVERWRITE 할 수 있습니다. 마치 멀티 파티션 테이블에 데이터를 저장하는 것처럼 말입니다.  
아래와 같이 suffixes 파라미터에 추가 파티션으로 사용할 컬럼 이름을 지정합니다. suffix가 여러 개인 경우(예: table_name__subpart1__subpart2)도 suffixes 파라미터에 해당 컬럼 이름들을 리스트로 넣어주면 INSERT OVERWRITE 할 수 있습니다.

In [24]:
from pydatafabric.gcp import bq_insert_overwrite, get_temp_table

sql = f"""
    SELECT *, 'subpart_1' as subpart
    FROM {PROJECT}.{DATASET}.{TABLE}
    WHERE dt = '{max_dt}'
""".format(**query_params)

result_table_name= get_temp_table()

bq_insert_overwrite(sql=sql, destination=result_table_name, suffixes=["subpart"], partition="dt", clustering_fields=["cust_id"])

destination: emart-datafabric.temp_1d.e19b0211_a352_4c24_99a5_c32b19111a04
total_rows: 25608
slot_secs: 3.601

destination: emart-datafabric.temp_1d.fb0255ec_75c3_48e9_9c59_e5d1254e8f3a__subpart_1
total_rows: 25608
slot_secs: 4.817



## BigQuery to Pandas
위에서 설명했듯이 BigQuery 쿼리 결과를 Pandas Dataframe으로 리턴받을 수 있습니다.
위의 IPython Magic 뿐만 아니라 emart 패키지에서 제공하는 메서드를 사용할 수도 있습니다.   
   
(참고 : Jupyter Notebook Cell에서 IPyhon Magic을 사용하여 SQL을 실행하는 경우 그 Cell에는 다른 Python 코드를 추가할 수 없습니다. Cell에 다른 Python 코드를 자유롭게 추가하려면 emart 패키지의 기능을 활용하시기 바랍니다.)


다음은 bq_to_pandas 메서드로 결과를 Pandas Dataframe에 저장하는 예입니다.

In [25]:
result_table_name

'emart-datafabric.temp_1d.fb0255ec_75c3_48e9_9c59_e5d1254e8f3a'

In [26]:
from pydatafabric.gcp import bq_to_pandas

pd_df = bq_to_pandas(f"""
    SELECT *
    FROM {PROJECT}.{DATASET}.{TABLE}
    WHERE dt = (SELECT MAX(dt) FROM {PROJECT}.{DATASET}.{TABLE})
""")

pd_df

unsupported operand type(s) for /: 'NoneType' and 'int'


Downloading: 100%|██████████| 25608/25608 [00:00<00:00, 30403.07rows/s]


,review_id,cust_id,gender_cd,gender_nm,age,agrde_cd_10_unit,store_cd,store_nm,score,comments,...,longitude,latitude,comments_point,image_point,thumb_point,action_cd,blind_flag,active_flag,tag_list,avg_tag_score
0,2208301432069315,02b7d95c81e0a0b16905e77547ac0b54881a0728438961...,2,여,62,60,1000,이마트 창동점,10,치즈 좋아하는데 쟁여 놔야 맘이 편안 그래서 또구입,...,127.046690683,37.651608368,10,10,0,003,N,Y,"332::1::8,332::2::6,332::3::6,332::4::6,332::5::6",6.000000000
1,2208301435131475,02b7d95c81e0a0b16905e77547ac0b54881a0728438961...,2,여,62,60,1000,이마트 창동점,10,모짜렐라 쟁여 놓은게 마침 떨어져서 구입 든든,...,127.046690683,37.651608368,10,10,0,003,N,Y,"332::1::6,332::2::6,332::3::6,332::4::6,332::5::4",6.000000000
2,2208301430288840,02b7d95c81e0a0b16905e77547ac0b54881a0728438961...,2,여,62,60,1000,이마트 창동점,10,담백하고 비리지 않은 가자미 국산이라네요 맛있게 먹을께용,...,127.046690683,37.651608368,10,10,0,003,N,Y,"153::1::6,153::2::6,153::3::6,153::4::6,153::5::6",6.000000000
3,2208300923271031,03c4f22b4d5b998916c8ae5358c68f9d1b89ff96a93bc4...,1,남,36,30,1000,이마트 창동점,10,저렴하고 맛있어요!!!!!!!!!,...,127.046690683,37.651608368,10,10,0,003,N,Y,"130::1::10,130::2::10,130::3::10,130::4::10,13...",10.000000000
4,2208300925053511,03c4f22b4d5b998916c8ae5358c68f9d1b89ff96a93bc4...,1,남,36,30,1000,이마트 창동점,10,맛있어요!!!!!!!!!,...,127.046690683,37.651608368,10,10,0,003,N,Y,"340::1::10,340::2::10,340::3::10,340::4::10,34...",10.000000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25603,2208301158538227,553497183b155892de10e30c28381f06406ce27f457341...,2,여,42,40,7407,토이킹덤 안성점,10,,...,127.147249595,36.994633421,0,0,0,000,N,Y,"542::1::10,542::2::10,542::3::10,542::4::10",10.000000000
25604,2208301522157501,14611f676a02617050fd2484f8ade5c8b9292111ea5e5e...,1,남,51,50,7709,일렉트로마트 스타필드 하남점,10,,...,127.223738447,37.545436142,0,0,0,000,N,Y,"534::1::10,534::2::10,534::3::6,534::4::8",9.000000000
25605,2208300440306896,ec8c3f6f08da9871739cd9273b886b001994de33b5167c...,1,남,48,40,7709,일렉트로마트 스타필드 하남점,10,,...,127.223738447,37.545436142,0,0,0,000,N,Y,"530::1::10,530::2::10,530::3::10,530::4::10",10.000000000
25606,2208300915461856,b4090107c89a0eb834d47e9a35fb80b74993db103bf1c0...,2,여,33,30,7714,일렉트로마트 스타필드 고양점,10,디자인 이쁘고 좋아요,...,126.894776182,37.646978959,10,0,0,001,N,Y,"530::1::10,530::2::10,530::3::10,530::4::10",10.000000000


## Pandas to BigQuery

Pandas Dataframe을 특정 BigQuery 테이블에 저장할 수 있습니다.  
다음과 같이 pandas_to_bq_table 메서드를 사용합니다.

In [88]:
import time
from pydatafabric.gcp import pandas_to_bq_table

dest_table = "dfm_sample_eapp_data_from_pandas"
print(f"저장할 테이블 : temp_1d.{dest_table}")

pandas_to_bq_table(pd_df, "temp_1d", dest_table)

get_bigquery_client().query(f"""
    SELECT *
    FROM temp_1d.{dest_table}
""").result()

저장할 테이블 : temp_1d.dfm_sample_eapp_data_from_pandas


22/10/05 13:08:06 INFO org.apache.spark.SparkEnv: Registering MapOutputTracker
22/10/05 13:08:06 INFO org.apache.spark.SparkEnv: Registering BlockManagerMaster
22/10/05 13:08:06 INFO org.apache.spark.SparkEnv: Registering BlockManagerMasterHeartbeat
22/10/05 13:08:06 INFO org.apache.spark.SparkEnv: Registering OutputCommitCoordinator
/usr/lib/spark/python/lib/pyspark.zip/pyspark/sql/pandas/conversion.py:289: UserWarning: createDataFrame attempted Arrow optimization because 'spark.sql.execution.arrow.pyspark.enabled' is set to true; however, failed by the reason below:
  Cannot specify a mask or a size when passing an object that is converted with the __arrow_array__ protocol.
Attempting non-optimization as 'spark.sql.execution.arrow.pyspark.fallback.enabled' is set to true.


데이터가 테이블에 저장되었는지 확인합니다.

In [89]:
%%bq

SELECT *
FROM emart-datafabric.temp_1d.dfm_sample_eapp_data_from_pandas

Downloading: 100%|██████████| 25608/25608 [00:00<00:00, 30681.88rows/s]

BigQuery execution took 2 seconds.


,review_id,cust_id,gender_cd,gender_nm,age,agrde_cd_10_unit,store_cd,store_nm,score,comments,...,longitude,latitude,comments_point,image_point,thumb_point,action_cd,blind_flag,active_flag,tag_list,avg_tag_score
0,2208301250200823,1129442e4826d1ad20db8dbf704ae0cfef1897112880e8...,2,여,46,40,1018,이마트 부천점,10,당도나식감이넣좋아요,...,126.782644897,37.484037379,10,0,0,001,N,Y,"110::1::10,110::2::10,110::3::10,110::4::10,11...",10.000000000
1,2208301511164743,146256e2ac7bcbf96258147d2f53f9aba5b1be767fe370...,2,여,69,60,1018,이마트 부천점,10,세일을 크게 해서 두 팩 샀습니다,...,126.782644897,37.484037379,10,0,0,001,N,Y,"111::1::10,111::2::10,111::3::10,111::4::10,11...",10.000000000
2,2208300903316888,205d023499cc9f50292187037f8b9507e3d78c2d0416d9...,2,여,63,60,1018,이마트 부천점,10,가격이 싸서 좋았어요품질도 그럭저럭 괜찮았어요,...,126.782644897,37.484037379,10,0,0,001,N,Y,"120::1::8,120::2::8,120::3::10,120::4::8,120::...",8.000000000
3,2208300033556742,848a65ff916644625e7a5f62686df24523fb787865b4fe...,2,여,68,60,1018,이마트 부천점,10,다른 아몬드는 팬에 덕어 먹었는데\r\n이름데로 구워서인지 그냥 먹고 있는데\r\n...,...,126.782644897,37.484037379,10,0,0,001,N,Y,"113::1::10,113::2::10,113::3::10,113::4::10,11...",10.000000000
4,2208300928376645,8bb2d74029eabf1027e0e382775a567444919b3ec87e04...,2,여,52,50,1018,이마트 부천점,10,맛있어서 재구매 합니다,...,126.782644897,37.484037379,10,0,0,001,N,Y,"113::1::10,113::2::10,113::3::10,113::4::10,11...",10.000000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25603,2208301505302932,7f2f06ab753d5630c5c4e3f4ad3e60ec0a9f616eb77877...,2,여,58,50,1063,이마트 마산점,10,시골에 필요해서 얼릉 구입했어요,...,128.569953950,35.197857791,10,0,0,001,N,Y,"430::1::10,430::2::10,430::3::10,430::4::10,43...",10.000000000
25604,2208301506513969,7f2f06ab753d5630c5c4e3f4ad3e60ec0a9f616eb77877...,2,여,58,50,1063,이마트 마산점,10,가격도 만족 잘쓸게요,...,128.569953950,35.197857791,10,0,0,001,N,Y,"473::1::10,473::2::10,473::3::10,473::4::10,47...",10.000000000
25605,2208301506231328,7f2f06ab753d5630c5c4e3f4ad3e60ec0a9f616eb77877...,2,여,58,50,1063,이마트 마산점,10,좋아요 항상 잘쓰고있네요,...,128.569953950,35.197857791,10,0,0,001,N,Y,"470::1::10,470::2::10,470::3::10,470::4::10,47...",10.000000000
25606,2208301504504116,7f2f06ab753d5630c5c4e3f4ad3e60ec0a9f616eb77877...,2,여,58,50,1063,이마트 마산점,10,너무 잘산것같네요 잘쓸게요,...,128.569953950,35.197857791,10,0,0,001,N,Y,"473::1::10,473::2::10,473::3::10,473::4::10,47...",10.000000000


## BigQuery to Spark  

BigQuery 데이터를 가져와서 Spark로 처리할 수 있습니다. BigQuery SQL 결과를 Spark Dataframe으로 변환한 후 이어서 데이터 처리가 가능합니다.

In [90]:
from pydatafabric.gcp import bq_to_df, get_spark

spark = get_spark()

spark_df = bq_to_df(f"""
    SELECT sku_cd
          ,sku_nm
          ,dt
    FROM temp_1d.{dest_table}
""", spark_session=spark)

22/10/05 13:08:58 INFO org.apache.spark.SparkEnv: Registering MapOutputTracker
22/10/05 13:08:58 INFO org.apache.spark.SparkEnv: Registering BlockManagerMaster
22/10/05 13:08:58 INFO org.apache.spark.SparkEnv: Registering BlockManagerMasterHeartbeat
22/10/05 13:08:58 INFO org.apache.spark.SparkEnv: Registering OutputCommitCoordinator


In [91]:
spark_df.show(10)

+-------------+------+----------+
|       sku_cd|sku_nm|        dt|
+-------------+------+----------+
|2500000039232|  근대|2022-08-30|
|2500000039232|  근대|2022-08-30|
|2500000039232|  근대|2022-08-30|
|2500000039232|  근대|2022-08-30|
|2500000039232|  근대|2022-08-30|
|2500000039232|  근대|2022-08-30|
|2500000039140|  깻순|2022-08-30|
|2332310000000|  돌김|2022-08-30|
|2500000110139|  봄동|2022-08-30|
|2500000110139|  봄동|2022-08-30|
+-------------+------+----------+
only showing top 10 rows



## Spark to BigQuery  

Spark으로 처리한 결과를 다시 BigQuery에 적재가 가능합니다.  
다음은 위에서 spark_df 변수에 저장한 Spark Dataframe을 다시 BigQuery에 적재하는 예제입니다.

1. 먼저 저장할 테이블을 BigQuery에 생성합니다. 여기서는 파티션된 테이블을 생성하고 특정 파티션에 Spark Dataframe을 저장할 것입니다.

In [92]:
from pydatafabric.gcp import df_to_bq_table

dest_dataset = "temp_1d"
partitioned_dest_table = f"dfm_sample_eapp_data_from_spark"

get_bigquery_client().query(f"""
    CREATE OR REPLACE TABLE {dest_dataset}.{partitioned_dest_table}
    (
        sku_cd STRING,
        sku_nm STRING,
        dt DATE
    )
    PARTITION BY dt
""").result()

print(f"생성된 테이블 : {dest_dataset}.{partitioned_dest_table}")

생성된 테이블 : temp_1d.dfm_sample_eapp_data_from_spark


2. Spark Dataframe을 BigQuery 테이블에 저장합니다. 여기서는 파티션 컬럼 타입이 Date이므로 타입 변환 작업을 해주었습니다. 이렇게 경우에 따라 타입 변환 작업이 필요한 경우가 있으므로 타입에 주의해주세요.

In [93]:
changed_df = spark_df.select("sku_cd", "sku_nm", spark_df.dt.cast("date"))
partition_dt = changed_df.head(1)[0].dt.strftime("%Y%m%d")
df_to_bq_table(df=changed_df,
               dataset=dest_dataset,
               table_name=partitioned_dest_table,
               partition=partition_dt,
               mode="overwrite")

3. BigQuery에 저장되었는지 확인합니다.

In [95]:
%%bq
    SELECT *
    FROM emart-datafabric.temp_1d.dfm_sample_eapp_data_from_spark

Downloading: 100%|██████████| 25608/25608 [00:00<00:00, 33595.49rows/s]

BigQuery execution took 2 seconds.


,sku_cd,sku_nm,dt
0,2500000039232,근대,2022-08-30
1,2500000039232,근대,2022-08-30
2,2500000039232,근대,2022-08-30
3,2500000039232,근대,2022-08-30
4,2500000039232,근대,2022-08-30
...,...,...,...
25603,2700000445259,진)슈틸루스터 탁상용 무선써큘레이터(베이,2022-08-30
25604,8801052091921,청정원맛선생 아귀청양고추국물내기티백72g,2022-08-30
25605,8801046331330,노브랜드보타니클래식퓨어라벤더바디워시1L,2022-08-30
25606,8801166239684,좋은느낌유기농무표백입는오버나이트중형8P,2022-08-30


In [96]:
spark.stop()